In [1]:
#import matplotlib.pyplot as plt
#import seaborn as sns


import pandas as pd
import numpy as np
import random as rnd

from sklearn.cross_validation import KFold, cross_val_score

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier



train_=pd.read_csv('../train_allcols.csv')
validate_=pd.read_csv('../validate_allcols.csv')
#test=pd.read_csv('../testwDSM.csv')

characters = pd.read_csv('finalcharacters.csv', sep=',')

train_.shape, validate_.shape, #test.shape

/Users/arc/Codes/DM-Lab/dmlab/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


((301619, 62), (148560, 62))

In [2]:
characters.shape


(9, 63)

In [5]:
train = train_.query('DSMCRIT < 14').sample(20000)
validate = validate_.query('DSMCRIT < 14').sample(10000)
#print train['DSMCRIT'].value_counts()
print train.shape

(20000, 62)


In [6]:
#train = train.query('SUB1 <= 10').query('SUB2 <= 10')
#validate = validate.query('SUB1 <= 10').query('SUB2 <= 10')

drop_list = ['DSMCRIT',  #'NUMSUBS'
             ]
drop_list_select = ['RACE', 'PREG', 'ARRESTS', 'PSYPROB', 'DETNLF', 'ETHNIC', 'MARSTAT', 'GENDER', 'EDUC'
                   ,'LIVARAG', 'EMPLOY', 'SUB3']

retain_list = ['RACE','PCPFLG','PRIMINC','LIVARAG','BENZFLG','HLTHINS','GENDER','ROUTE3','PRIMPAY',
               'MARSTAT','PSYPROB','ROUTE2','EMPLOY','SUB2','FRSTUSE3','FREQ3','FRSTUSE2','OTHERFLG',
               'EDUC','FREQ2','FREQ1','YEAR',
               'PSOURCE','DETCRIM','DIVISION','REGION','NOPRIOR','NUMSUBS','ALCDRUG',
               'METHUSE','FRSTUSE1','AGE','COKEFLG','OPSYNFLG','IDU','SERVSETA','ROUTE1','MARFLG',
               'MTHAMFLG','HERFLG',
               'ALCFLG','SUB1']
X_train = train[retain_list]

X_c = characters[retain_list]
Y_c = characters["DSMCRIT"]
#X_train = train.drop(drop_list + drop_list_select, axis=1)
Y_train = train["DSMCRIT"]
#X_validate = validate.drop(drop_list + drop_list_select, axis=1)
#Y_validate = validate["DSMCRIT"]
#X_test  = test.drop(drop_list, axis=1)
X_train.shape, #X_validate.shape, #X_test.shape



((20000, 42),)

In [7]:
print X_train.columns.tolist()

['RACE', 'PCPFLG', 'PRIMINC', 'LIVARAG', 'BENZFLG', 'HLTHINS', 'GENDER', 'ROUTE3', 'PRIMPAY', 'MARSTAT', 'PSYPROB', 'ROUTE2', 'EMPLOY', 'SUB2', 'FRSTUSE3', 'FREQ3', 'FRSTUSE2', 'OTHERFLG', 'EDUC', 'FREQ2', 'FREQ1', 'YEAR', 'PSOURCE', 'DETCRIM', 'DIVISION', 'REGION', 'NOPRIOR', 'NUMSUBS', 'ALCDRUG', 'METHUSE', 'FRSTUSE1', 'AGE', 'COKEFLG', 'OPSYNFLG', 'IDU', 'SERVSETA', 'ROUTE1', 'MARFLG', 'MTHAMFLG', 'HERFLG', 'ALCFLG', 'SUB1']


In [8]:
#one hot
from sklearn import preprocessing

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(X_train)

# 3. Transform
onehotlabels = enc.transform(X_train).toarray()
X_train = onehotlabels

#onehotlabels = enc.transform(X_validate).toarray()
#X_validate = onehotlabels


X_char = enc.transform(X_c).toarray()

print X_train.shape, X_char.shape#X_validate.shape

(20000, 241) (9, 241)


In [9]:
#kfold
kf = 3

In [10]:
# Logistic Regression
logreg = LogisticRegression(n_jobs=-1)
logreg.fit(X_train, Y_train)
#Y_pred = logreg.predict(X_test)

l_acc_log = cross_val_score(logreg, X_train, Y_train, cv=kf)
acc_log = round(np.mean(l_acc_log), 3)

l_acc_log = ['%.3f' % elem for elem in l_acc_log]
print l_acc_log
print acc_log

/Users/arc/Codes/DM-Lab/dmlab/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


['0.714', '0.722', '0.726']
0.72


In [11]:
# Random Forest (slow)

random_forest = RandomForestClassifier(n_estimators=200, max_depth=20, n_jobs=-1)
random_forest.fit(X_train, Y_train)
#Y_pred = random_forest.predict(X_test)

l_acc_random_forest = cross_val_score(random_forest, X_train, Y_train, cv=kf)
acc_random_forest = round(np.mean(l_acc_random_forest), 3)

l_acc_random_forest = ['%.3f' % elem for elem in l_acc_random_forest]
print l_acc_random_forest
print acc_random_forest

['0.730', '0.731', '0.737']
0.732


In [12]:
yp_char = logreg.predict_proba(X_char)
print yp_char
print Y_c

[[  2.48026082e-05   8.47060833e-04   6.78041506e-05   4.10089433e-05
    8.86824597e-01   5.84246877e-03   6.94242131e-03   1.85640337e-02
    6.21787191e-02   1.55748832e-02   2.86768461e-03   6.42094040e-05
    1.54525456e-04   5.78119779e-06]
 [  7.79222212e-06   6.23432747e-05   1.44888558e-02   2.02577185e-07
    1.30143089e-03   1.51093327e-03   3.10327924e-02   3.11539401e-01
    4.49365022e-02   5.40482552e-02   5.16822930e-01   1.12331650e-02
    6.88856965e-03   6.12682674e-03]
 [  3.06623318e-05   7.40978843e-06   9.80283801e-03   4.83150937e-07
    1.27918394e-02   3.29850759e-02   3.03036732e-03   3.92250123e-03
    8.39466650e-01   1.11421716e-03   8.72364646e-04   4.14389562e-02
    5.45226746e-02   1.39599315e-05]
 [  5.10315622e-04   1.12365131e-05   1.30565880e-01   9.48668135e-07
    1.95517531e-02   4.67764386e-03   3.22774928e-02   5.78464375e-01
    1.70526209e-02   2.78047664e-03   1.95034210e-01   2.02081701e-03
    1.00335719e-03   1.60488725e-02]
 [  4.878665

In [13]:
yp_char = random_forest.predict_proba(X_char)
print yp_char

[[  6.17309040e-04   1.32862105e-02   3.32028908e-03   8.61583413e-03
    9.02599318e-01   2.53095823e-04   5.27687206e-03   2.55666289e-02
    9.28412369e-03   2.74258461e-02   2.27368696e-03   1.41466608e-03
    0.00000000e+00   6.61200171e-05]
 [  5.50901876e-03   2.89017341e-05   5.31847134e-04   0.00000000e+00
    6.49768875e-03   7.87437034e-04   8.52256261e-03   5.08398525e-01
    7.15432092e-02   4.44634516e-02   3.51782458e-01   1.55221992e-03
    3.82679535e-04   0.00000000e+00]
 [  1.00000000e-02   0.00000000e+00   1.82284382e-02   5.00000000e-03
    1.24808715e-01   2.78214057e-01   1.05048077e-02   1.50000000e-02
    4.24851081e-01   2.66241112e-02   0.00000000e+00   4.12132353e-02
    4.55555556e-02   0.00000000e+00]
 [  1.72306637e-03   2.02020202e-05   8.08009915e-03   0.00000000e+00
    7.13567480e-02   6.37676418e-04   3.15345142e-02   6.40527506e-01
    4.13241904e-02   2.02808985e-02   1.75776683e-01   1.65137002e-03
    5.64941301e-03   1.43763271e-03]
 [  5.297146